# FUNCTIONS

In [ ]:
def find_nearest3(array, values):
    values = np.atleast_1d(values)
    indices = np.abs(np.int64(np.subtract.outer(array, values))).argmin(0)
    out = array[indices]
    return indices

In [ ]:
def center_array_indices(x_in, N):
    l = len(x_in)
    M = np.round((N-1)/2)+1
    x_out = np.array([np.nan]*l)
    x_out[int(N-M):int(l-M)] = x_in[N:l]
    return x_out

In [ ]:
def interpholes(x_in, y_in, x_out):
    ff = np.where((np.isfinite(y_in)))[0]
    y_out = np.interp(x_out, x_in[ff], y_in[ff])   
    return y_out

In [1]:
def beaning_weekly(y_in, marker_range, marker1, marker2): 
    l = len(marker_range)
    x_out = np.empty(l*52) * np.NaN
    x_out2 = np.empty(l*52) * np.NaN
    x_out3 = np.empty(l*52) * np.NaN
    x_out4 = np.empty(l*52) * np.NaN
    y_out = np.empty(l*52) * np.NaN 
    k = -1
    for i in range(0,l): 
        for w in range(0,53):
            k = k+1
            v = marker_range[i]
            x_out2[k] = v 
            x_out3[k] = w
            x_out4[k] = k
            x_out[k] = v + w/53
            fiw = np.where((marker1 == v) & (marker2 == w))[0]
            if len(fiw)>0:
                y_out[k] = np.nanmean(y_in[fiw])
    return x_out, x_out2, x_out3, x_out4, y_out

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
def beaning_plus(x_in, marker_range, marker):
    l = len(marker_range)
    x_out = np.empty(l) * np.NaN
    xx_out = np.empty(l) * np.NaN
    std_out = np.empty(l) * np.NaN
    stde_out = np.empty(l) * np.NaN
    N_out = np.empty(l) * np.NaN
    
    for i in range(0,l): 
        v = marker_range[i]
        f = np.where((marker == v))[0]
        if len(f)>0:
            N_out[i] = len(f)
            std_out[i] = np.nanstd(x_in[f])
            stde_out[i] = np.nanstd(x_in[f])/np.sqrt(len(f))
            x_out[i] = np.nanmean(x_in[f])
            xx_out[i] = np.nanmedian(x_in[f])
    return x_out, std_out, stde_out, N_out, xx_out

In [ ]:
def beaning(x_in, marker_range, marker):
    l = len(marker_range)
    x_out = np.empty(l) * np.NaN
    for i in range(0,l): 
        v = marker_range[i]
        f = np.where((marker == v))[0]
        if len(f)>0:
            x_out[i] = np.nanmean(x_in[f])
    return x_out

def integrate(X, z1, z2):
    sxx = X.shape
    sx1 = sxx[0]
    sx2 = sxx[1]
    x_out  = np.empty(sx2) * np.NaN
    for i in range(0,sx2): 
        z = np.arange(int(z1[i]),int(z2[i])+1)
        v = X[z,i]   
        fv = np.where((np.isfinite(v)))[0]
        if len(np.isfinite(v) == True) > 0:
            x_out[i] = np.nansum(X[fv,i]) 
    x_out[x_out==0] = np.nan 
    return x_out

In [ ]:
def datenum(d):
    return 366 + d.toordinal() + (d - dt.fromordinal(d.toordinal())).total_seconds()/(24*60*60)
#d = dt.strptime('2019-2-1 12:24','%Y-%m-%d %H:%M')
#datenum(d)


In [ ]:
from scipy.stats import t

def make_linregress(y_in, marker_1, marker_2, range_marker_1, range_marker_2):
    l = len(range_marker_1)
    slpe = np.array([np.NaN] * l)
    rsqu = np.array([np.NaN] * l)
    pval = np.array([np.NaN] * l)
    stde = np.array([np.NaN] * l)
    intr = np.array([np.NaN] * l)
    slpe_ci = np.array([np.NaN] * l)
    for m in range(range_marker_1[0],range_marker_1[l-1]+1):
        fm = np.where((marker_1 == m))[0]
        y = beaning(y_in[fm], range_marker_2, marker_2[fm]) #  - - - y
        x = range_marker_2
        ff = np.where(np.isfinite(y))[0]
        if len(ff)>0:
            X = np.reshape(x, (len(x), 1))
            model = LinearRegression()
            model.fit(X, y)
            trend = model.predict(X) #  - - - - - - - - - - - - - - - - trend
            detrended = [y[i]-trend[i] for i in range(0, len(y))]
            res = stats.linregress(x,y) #  - - - - - - - - - - - - - - - stats
            trend = res.slope * x + res.intercept
            slpe[m-1] = res.slope
            rsqu[m-1] = res.rvalue**2
            pval[m-1] = res.pvalue
            stde[m-1] = res.stderr
            intr[m-1] = res.intercept
            #
            tinv = lambda p, df: abs(t.ppf(p/2, df))
            ts = tinv(0.05, len(x)-2)
            slpe_ci[m-1] = ts*res.stderr
        
    return slpe, rsqu, pval, stde, intr, slpe_ci

In [1]:
def make_split_linregress(y_in, marker_1, val_markers):
    l = len(val_markers)-1
    slpe = np.array([np.NaN] * l)
    rsqu = np.array([np.NaN] * l)
    pval = np.array([np.NaN] * l)
    stde = np.array([np.NaN] * l)
    intr = np.array([np.NaN] * l)
    slpe_ci = np.array([np.NaN] * l)
    
    for seg in range(0,l):
        # print(seg)
        f = np.where((marker_1 >= val_markers[seg]) & (marker_1 < val_markers[seg+1]))[0]
        x = marker_1[f]
        y = y_in[f]
        ff = np.where(np.isfinite(y))[0]
        if len(ff)>0:
            x = x[ff]
            y = y[ff]
            X = np.reshape(x, (len(x), 1))
            model = LinearRegression()
            model.fit(X, y)
            trend = model.predict(X) #  - - - - - - - - - - - - - - - - trend
            detrended = [y[i]-trend[i] for i in range(0, len(y))]
            res = stats.linregress(x,y) #  - - - - - - - - - - - - - - - stats
            trend = res.slope * x + res.intercept
            slpe[seg] = res.slope
            rsqu[seg] = res.rvalue**2
            pval[seg] = res.pvalue
            stde[seg] = res.stderr
            intr[seg] = res.intercept
            #
            tinv = lambda p, df: abs(t.ppf(p/2, df))
            ts = tinv(0.05, len(x)-2)
            slpe_ci[seg] = ts*res.stderr       
    return slpe, rsqu, pval, stde, intr, slpe_ci, y_in

In [ ]:

# -*- coding: utf-8 -*-
#
# windstress.py
#
# purpose:
# author:   Filipe P. A. Fernandes
# e-mail:   ocefpaf@gmail
# web:      http://ocefpaf.github.io/
# created:  21-Aug-2013
# modified: Mon 21 Jul 2014 12:16:28 PM BRT
#
# obs:
#

import numpy as np

# from .constants import kappa, Charnock_alpha, g, R_roughness

# ---- meteorological constants
kappa = 0.4  # NOTE: 0.41
""" von Karman's constant """

charn = Charnock_alpha = 0.011  # NOTE: 0.018
""" Charnock constant. For determining roughness length at sea given friction
velocity, used in Smith formulas for drag coefficient and also in Fairall and
Edson. Ese alpha = 0.011 for open-ocean and alpha = 0.018 for fetch-limited
(coastal) regions."""

R_roughness = 0.11
""" limiting roughness Reynolds for aerodynamically smooth flow """



#from .atmosphere import visc_air

def visc_air(Ta):
    """Computes the kinematic viscosity of dry air as a function of air
    temperature
    Parameters
    ----------
    Ta : array_like
         air temperature [:math:`^\\circ` C]
    Returns
    -------
    visa : array_like
           [m :sup:`2` s :sup:`-1`]
    See Also
    --------
    hfbulktc, cdn
    sw.visc
    Notes
    -----
    sw.visc from python seawater package
    Examples
    --------
    >>> from airsea import atmosphere as asea
    >>> asea.visc_air([[0.1, 5., 15],[22.8, 28.9, 31.4]])
    array([[  1.32686758e-05,   1.36964784e-05,   1.45857532e-05],
           [  1.52942886e-05,   1.58573695e-05,   1.60903922e-05]])
    References
    ----------
    .. [1] Andreas (1989), CRREL Report 89-11.
    Modifications: Original from COARE 3.0
    11/26/2010: Filipe Fernandes, Python translation.
    """
    # convert input to numpy array
    Ta = np.asarray(Ta)

    visa = 1.326e-5 * (1 + 6.542e-3 * Ta + 8.301e-6 * Ta ** 2 - 4.84e-9 *
                       Ta ** 3)
    return visa



def cdn(sp, z, drag='largepond', Ta=10):
    """Computes neutral drag coefficient.
    Methods available are: Large & Pond (1981),  Vera (1983) or Smith (1988)
    Parameters
    ----------
    sp : array_like
         wind speed [m s :sup:`-1`]
    z : float, array_like
        measurement height [m]
    drag : str
           neutral drag by:
           'largepond' <-- default
           'smith'
           'vera'
    Ta : array_like, optional for drag='smith'
         air temperature [:math:`^\\circ` C]
    Returns
    -------
    cd : float, array_like
         neutral drag coefficient at 10 m
    u10 : array_like
          wind speed at 10 m [m s :sup:`-1`]
    See Also
    --------
    stress, spshft, visc_air
    Notes
    -----
    Vera (1983): range of fit to data is 1 to 25 [m s :sup:`-1`].
    Examples
    --------
    >>> from airsea import windstress as ws
    >>> ws.cdn([10., 0.2, 12., 20., 30., 50.], 10)
    (array([ 0.00115,  0.00115,  0.00127,  0.00179,  0.00244,  0.00374]),
     array([ 10. ,   0.2,  12. ,  20. ,  30. ,  50. ]))
    >>> ws.cdn([10., 0.2, 12., 20., 30., 50.], 15, 'vera')
    (array([ 0.00116157,  0.01545237,  0.00126151,  0.00174946,  0.00242021,
            0.00379521]),
     array([  9.66606155,   0.17761896,  11.58297824, 19.18652915,
            28.5750255 ,  47.06117334]))
    >>> ws.cdn([10., 0.2, 12., 20., 30., 50.], 20, 'smith', 20.)
    (array([ 0.00126578,  0.00140818,  0.00136533,  0.00173801,  0.00217435,
            0.00304636]),
     array([  9.41928554,   0.18778865,  11.27787697,  18.65250005,
            27.75712916,  45.6352786 ]))
    References
    ----------
    .. [1] Large and Pond (1981), J. Phys. Oceanog., 11, 324-336.
    .. [2] Smith (1988), J. Geophys. Res., 93, 311-326.
    .. [3] E. Vera (1983) FIXME eqn. 8 in Large, Morzel, and Crawford (1995),
    J. Phys. Oceanog., 25, 2959-2971.
    Modifications: Original from AIR_SEA TOOLBOX, Version 2.0
    03-08-1997: version 1.0
    08-26-1998: version 1.1 (vectorized by RP)
    08-05-1999: version 2.0
    11-26-2010: Filipe Fernandes, Python translation.
    """
    # convert input to numpy array
    sp, z, Ta = np.asarray(sp), np.asarray(z), np.asarray(Ta)

    tol = 0.00001  # Iteration end point.

    if drag == 'largepond':
        a = np.log(z / 10.) / kappa  # Log-layer correction factor.
        u10o = np.zeros(sp.shape)
        cd = 1.15e-3 * np.ones(sp.shape)
        u10 = sp / (1 + a * np.sqrt(cd))
        ii = np.abs(u10 - u10o) > tol

        while np.any(ii):
            u10o = u10
            cd = (4.9e-4 + 6.5e-5 * u10o)  # Compute cd(u10).
            cd[u10o < 10.15385] = 1.15e-3
            u10 = sp / (1 + a * np.sqrt(cd))  # Next iteration.
            # Keep going until iteration converges.
            ii = np.abs(u10 - u10o) > tol

    elif drag == 'smith':
        visc = visc_air(Ta)

        # Remove any sp==0 to prevent division by zero
        # i = np.nonzero(sp == 0)
        # sp[i] = 0.1 * np.ones(len(i)) FIXME

        # initial guess
        ustaro = np.zeros(sp.shape)
        ustarn = 0.036 * sp

        # iterate to find z0 and ustar
        ii = np.abs(ustarn - ustaro) > tol
        while np.any(ii):
            ustaro = ustarn
            z0 = Charnock_alpha * ustaro ** 2 / g + R_roughness * visc / ustaro
            ustarn = sp * (kappa / np.log(z / z0))
            ii = np.abs(ustarn - ustaro) > tol

        sqrcd = kappa / np.log(10. / z0)
        cd = sqrcd ** 2
        u10 = ustarn / sqrcd
    elif drag == 'vera':
        # constants in fit for drag coefficient
        A = 2.717e-3
        B = 0.142e-3
        C = 0.0764e-3

        a = np.log(z / 10.) / kappa  # Log-layer correction factor.
        # Don't start iteration at 0 to prevent blowups.
        u10o = np.zeros(sp.shape) + 0.1
        cd = A / u10o + B + C * u10o
        u10 = sp / (1 + a * np.sqrt(cd))

        ii = np.abs(u10 - u10o) > tol
        while np.any(ii):
            u10o = u10
            cd = A / u10o + B + C * u10o
            u10 = sp / (1 + a * np.sqrt(cd))  # Next iteration.
            # Keep going until iteration converges.
            ii = np.abs(u10 - u10o) > tol
    else:
        print('Unknown method')  # FIXME: raise a proper python error.

    return cd, u10


def spshft(sp, z1, z2, drag='largepond', Ta=10.):
    """Adjusts wind speed from height z1 to z2. Methods available are: Large &
    Pond (1981),  Vera (1983) or Smith (1988).
    Parameters
    ----------
    sp : array_like
          wind speed [m s :sup:`-1`]
    z1 : float
         measurement height [m]
    z2 : float
         desired height [m]
    drag : str
           neutral drag by:
           'largepond' <-- default
           'smith'
           'vera'
    Ta : array_like
         air temperature [:math:`^\\circ` C]
    Returns
    -------
    sp_adj : array_like
          predicted wind speed [m s :sup:`-1`]
    ustar : array_like
            friction velocity [m s :sup:`-1`]
    See Also
    --------
    cdn
    Examples
    --------
    >>> from airsea import windstress as ws
    >>> ws.spshft([10., 0.2, 12., 20., 30., 50.], 10, 10)[0]
    array([ 10. ,   0.2,  12. ,  20. ,  30. ,  50. ])
    >>> from airsea import windstress as ws
    >>> ws.spshft([10., 0.2, 12., 20., 30., 50.], 10, 8, 'smith', 20)
    (array([  9.79908171,   0.19583568,  11.74922628,  19.52618419,
            29.20068179,  48.40456082]), array([ 0.3601597 ,  0.00746483,  0.44952896,  0.84934708,  1.43283229,
            2.8599333 ]))
    >>> ws.spshft([10., 0.2, 12., 20., 30., 50.], 15, 10, 'vera')
    (array([  9.66606155,   0.17761896,  11.58297824,  19.18652915,
            28.5750255 ,  47.06117334]), array([ 0.32943742,  0.02207938,  0.41140089,  0.80250639,  1.40576781,
            2.89921535]))
    References
    ----------
    .. [1] Large and Pond (1981), J. Phys. Oceanog., 11, 324-336.
    .. [2] Smith (1988), J. Geophys. Res., 93, 311-326.
    .. [3] E. Vera (1983) FIXME eqn. 8 in Large, Morzel, and Crawford (1995),
    J. Phys. Oceanog., 25, 2959-2971.
    Modifications: Original from AIR_SEA TOOLBOX, Version 2.0
    03-08-1997: version 1.0
    08-27-1998: version 1.1 (revised to use cdn* efficiently by RP)
    08-05-1999: version 2.0
    11-26-2010: Filipe Fernandes, Python translation.
    """

    z1, z2 = np.asarray(z1), np.asarray(z2)
    sp, Ta = np.asarray(sp), np.asarray(Ta)

    # Find cd and ustar.
    if drag == 'largepond':
        cd10, sp10 = cdn(sp, z1, 'largepond')
    elif drag == 'smith':
        cd10, sp10 = cdn(sp, z1, 'smith', Ta)
    elif drag == 'vera':
        cd10, sp10 = cdn(sp, z1, 'vera')
    else:
        print('Unknown method')  # FIXME: raise a proper python error!

    ustar = np.sqrt(cd10) * sp10
    sp_adj = sp10 + ustar * np.log(z2 / 10.) / kappa
    return sp_adj, ustar


def stress(sp, z=10., drag='largepond', rho_air=1.22, Ta=10.):
    """Computes the neutral wind stress.
    Parameters
    ----------
    sp : array_like
         wind speed [m s :sup:`-1`]
    z : float, array_like, optional
        measurement height [m]
    rho_air : array_like, optional
           air density [kg m :sup:`-3`]
    drag : str
           neutral drag by:
           'largepond' <-- default
           'smith'
           'vera'
    Ta : array_like, optional
         air temperature [:math:`^\\circ` C]
    Returns
    -------
    tau : array_like
          wind stress  [N m :sup:`-2`]
    See Also
    --------
    cdn
    Examples
    --------
    >>> from airsea import windstress as ws
    >>> ws.stress([10., 0.2, 12., 20., 30., 50.], 10)
    array([  1.40300000e-01,   5.61200000e-05,   2.23113600e-01,
             8.73520000e-01,   2.67912000e+00,   1.14070000e+01])
    >>> kw = dict(rho_air=1.02, Ta=23.)
    >>> ws.stress([10., 0.2, 12., 20., 30., 50.], 15, 'smith', **kw)
    array([  1.21440074e-01,   5.32531576e-05,   1.88322389e-01,
             6.62091968e-01,   1.85325310e+00,   7.15282267e+00])
    >>> ws.stress([10., 0.2, 12., 20., 30., 50.], 8, 'vera')
    array([  1.50603698e-01,   7.16568379e-04,   2.37758830e-01,
             9.42518454e-01,   3.01119044e+00,   1.36422742e+01])
    References
    ----------
    .. [1] Large and Pond (1981), J. Phys. Oceanog., 11, 324-336.
    .. [2] Smith (1988), J. Geophys. Res., 93, 311-326.
    .. [3] E. Vera (1983) FIXME eqn. 8 in Large, Morzel, and Crawford (1995),
    J. Phys. Oceanog., 25, 2959-2971.
    Modifications: Original from AIR_SEA TOOLBOX, Version 2.0
    03-08-1997: version 1.0
    08-26-1998: version 1.1 (revised by RP)
    04-02-1999: versin 1.2 (air density option added by AA)
    08-05-1999: version 2.0
    11-26-2010: Filipe Fernandes, Python translation.
    """
    z, sp = np.asarray(z), np.asarray(sp)
    Ta, rho_air = np.asarray(Ta), np.asarray(rho_air)

    # Find cd and ustar.
    if drag == 'largepond':
        cd, sp = cdn(sp, z, 'largepond')
    elif drag == 'smith':
        cd, sp = cdn(sp, z, 'smith', Ta)
    elif drag == 'vera':
        cd, sp = cdn(sp, z, 'vera')
    else:
        print('Unknown method')  # FIXME: raise a proper python error

    tau = rho_air * (cd * sp ** 2)

    return tau


In [ ]:
def average_on_unique(x_in, marker):
    ux, iu = np.unique(marker, return_index=True)
    l = len(iu)
    x_out = np.empty(l) * np.NaN
    for i in range(0,l): 
        f = np.where((marker == ux[i]))[0]
        if len(f)>0:
            x_out[i] = np.nanmean(x_in[f])
    return x_out, ux, iu